# Challenge 5: 127 量子ビットを使う

これまでのChallengeが解けたとのこと、おめでとうございます！Dyanmic circuitをうまく使えることがわかりましたね。
ここまで来たご褒美として、127量子ビットデバイスの1つを使うことができます。


注意事項として、Challengeの完了でバッジを獲得するには、5つのLabのうち4つを終了している必要があります。このように決定した理由は、このLabが (a)技術的に困難であり、 (b)当社の最先端のハードウェアを使用することが分かっているためで、意図的にこのように決定しました。100量子ビットを超えるデバイスを一般に公開すると、ジョブの実行に大きな遅れが生じる可能性があるというのが実情です。Challengeの終了までにジョブを成功させることができない可能性もあります（そうでないことを祈りますが！）。そのようなことがないように、私たちは舞台裏でできる限りのことをしていますが、しかし、今年のChallengeは過去最高の参加者数を記録しています。親切に、何度も回路を提出し、キューを詰まらせないようにしてください。

まず、深呼吸してみましょう。では、始めましょう！

実際の物理的なデバイスを扱うには、それなりの困難が伴うことは、すでにお分かりいただけたと思います。
多数の量子ビットを使用する場合は、経路が長くなり、多くのエラーが発生しないようにする必要があるため、なおさらです。ここでは、ibm_sherbrookをデバイスとして使用します。

この課題では、127量子ビットの完全なエンタングル状態、いわゆるGHZ状態を賢い方法で準備してもらいます。

その後、エラー訂正の知識を127量子ビットのGHZ状態に適用し、54量子ビットのGHZ状態を作成するようガイドします。そのために、54量子ビットGHZ状態には偶数量子ビットを使用し、奇数量子ビットはスタビライザーとして使用することにします。

*注：奇数量子ビットと偶数量子ビットの使い方が違っても、同じレジスターを使うことにします。これは、特にDepthを手動で最適化する場合に、最初の127量子ビットのGHZ状態を簡単に作成できるようにするためです。*

In [ ]:
# Importing all the parts we need
from typing import List, Optional

from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.result import marginal_counts

import warnings

warnings.filterwarnings("ignore")

import math

pi = math.pi

# Preparing registers
quantum_register = QuantumRegister(127)
classical_register = ClassicalRegister(127)

# For simplicity we map the physical qubits to the logical qubits directly using the same number.
initial_layout = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    102,
    103,
    104,
    105,
    106,
    107,
    108,
    109,
    110,
    111,
    112,
    113,
    114,
    115,
    116,
    117,
    118,
    119,
    120,
    121,
    122,
    123,
    124,
    125,
    126,
]
# The "even"" qubits will be used for the 54 qubit GHZ-state
ghz_qubits = [
    0,
    2,
    4,
    6,
    8,
    10,
    12,
    18,
    20,
    22,
    24,
    26,
    28,
    30,
    32,
    37,
    39,
    41,
    43,
    45,
    47,
    49,
    51,
    56,
    58,
    60,
    62,
    64,
    66,
    68,
    70,
    75,
    77,
    79,
    81,
    83,
    85,
    87,
    89,
    94,
    96,
    98,
    100,
    102,
    104,
    106,
    108,
    114,
    116,
    118,
    120,
    122,
    124,
    126,
]
# The "odd" qubits will be used as the stabilizers
stabilizer_qubits = [
    1,
    3,
    5,
    7,
    9,
    11,
    14,
    15,
    16,
    17,
    19,
    21,
    23,
    25,
    27,
    29,
    31,
    33,
    34,
    35,
    36,
    38,
    40,
    42,
    44,
    46,
    48,
    50,
    52,
    53,
    54,
    55,
    57,
    59,
    61,
    63,
    65,
    67,
    69,
    71,
    72,
    73,
    74,
    76,
    78,
    80,
    82,
    84,
    86,
    88,
    90,
    91,
    92,
    93,
    95,
    97,
    99,
    101,
    103,
    105,
    107,
    109,
    110,
    111,
    112,
    115,
    117,
    119,
    121,
    123,
    125,
]

## Step 1: 大きなGHZ状態を作る

多くの量子アルゴリズムでは、複数の量子ビットを互いにエンタングルする必要があります。
アルゴリズムでよく使われる状態の1つに、一般化されたGHZ状態があり、 $(\alpha |00...0\rangle + \beta |11...1\rangle)$ このとき $\alpha = \beta$ という状態です。

GHZ状態は完全にエンタングルされており、任意の数の量子ビットに一般化することができます。ここでは、これを用いて127量子ビットのGHZ状態を作ることにします！
**何個の量子ビットによるGHZ状態であっても、GHZ状態は測定されると、すべての量子ビットが0か1かのどちらかになっています。**

ここでのchallengeは、127個の量子ビットを理論的にエンタングルさせることではなく、 _実際のデバイス上で_ エンタングルさせることです。つまり、デバイスのレイアウトを考慮する必要があります（例えば、互いに直接接続されていない量子ビットの間でCXゲートを使用しないなど）。また、ノイズを減らすために、回路の深さをできるだけ小さくするようにする必要があります。

### Exercise 1

アダマールゲートとCNOTゲートだけを使って、127量子ビットのデバイス `ibm_sherbrook` にGHZ状態を生成してください。できるだけ回路のDepthが小さくなるようにしてみてください。

*ヒント：すべての量子ビットが互いに接続されている理想的なデバイスに対して、（最小のDepthで）どのように行うかを最初に考えることが助けになることがあります。*

In [ ]:
def generate_ghz127():
    qc = QuantumCircuit(quantum_register, classical_register)

    ####### your code goes here #######

    return qc


ghz_circuit = generate_ghz127()
print(ghz_circuit.depth())

In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5a

grade_ex5a(ghz_circuit)

## Step 2: サイズを半分にする

ここで、作成したGHZ状態を利用して、偶数量子ビットを使った縮小GHZ状態を形成したいと思います。そのためには、GHZ状態を崩壊させることなく測定できるように、奇数量子ビットのエンタングルメントを解消させる必要があります。

量子ビットをアンエンタングルしたいので、Lab４でやったのと同じように、 **スタビライザー** を生成する方法で行います。スタビライザーは、接続されている両方の量子ビットが同じ値であれば0になり、異なる値であれば1になるはずです。

このようなことをすることに何の意味があるのでしょうか？鋭い観察者はすでに気づいているかもしれませんが、このスタビライザーを使って、縮小されたGHZ状態にエラー訂正を適用することができるのです。

### Exercise 2

スタビライザーを作るために、作られたGHZ状態の奇数量子ビットをアンエンタングルしてください。ここでも、回路のDepthをできるだけ小さくすることに挑戦できます。

*ヒント：上記のGHZ状態の作り方を考慮に入れてください。アンエンタングリングの方法は、その作成方法に特化したものです。*

In [ ]:
def deentangle_qubits():
    qc = QuantumCircuit(quantum_register, classical_register)

    ####### your code goes here #######

    return qc


unentangle_circuit = deentangle_qubits()
print(unentangle_circuit.depth())

complete_circuit = ghz_circuit.compose(unentangle_circuit)

In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5b

grade_ex5b(complete_circuit)

よくできました。さて、すべての奇数量子ビットは偶数量子ビットとエンタングルしていないので、GHZ状態を壊すことなく測定することができます。

In [ ]:
# Measuring stabilizers this can also be used in post processing to see what went wrong.


def measure_stabilizers():
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(stabilizer_qubits, stabilizer_qubits)
    return qc


stabilizer_circuit = measure_stabilizers()

これらの測定結果は、54量子ビットのGHZ状態を改善するために使用することができますが、このアイデアは後で検討します。

奇数量子ビットを測定した後、それらをリセットし、奇数量子ビットを潜在的な補助量子ビットとして使って、54量子ビットのGHZ状態をアルゴリズムに使用することができます。しかし、GHZ状態を使う前に、その実力を試してみましょう。

54量子ビットのGHZ状態を測定する必要があります！

In [ ]:
# Measuring the GHZ qubits


def measure_ghz():
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(ghz_qubits, ghz_qubits)
    return qc


measure_circuit = measure_ghz()

では、すべてをまとめて、テストできるようにしましょう。

In [ ]:
# Everything together

simple_ghz = (
    ghz_circuit.compose(unentangle_circuit)
    .compose(stabilizer_circuit)
    .compose(measure_circuit)
)

## Step 3: 準備し、実デバイスで実行する

GHZ状態を実際のデバイス上で実行する準備が整いましたので、`ibm_sherbrooke`の出番です。

127個の量子ビットを扱ったことのある人は、世界でもそう多くはないでしょう。あなたはすぐに、自分がその一人であると言える数少ない勇者の一人になるでしょう。量子コンピューターにとっては小さな一歩、あなたの履歴書にとっては大きな一歩です。

冗談です :-P

-----

まず、必要なものをすべて準備します。


In [ ]:
# Importing provider and getting access to the system
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile


provider = IBMProvider()


hub = "YOUR_HUB"
group = "YOUR_GROUP"
project = "YOUR_PROJECT"

backend_name = "ibm_sherbrooke"
backend = provider.get_backend(backend_name, instance=f"{hub}/{group}/{project}")

number_of_shots: int = 1024

まず、実際のデバイス用にトランスパイルすることから始めます。これは、あまり大きな変化はないはずです。なぜなら、回路を作るときに物理的なデバイスを念頭に置き、存在する接続だけを使用するようにした（はず）だからです。

もしそうでなかったなら、今すぐ戻って修正するか、Discordに飛び込んで、そのプロセスをどのように考えるかについて他の人と話し合う良い機会だと思います。

In [ ]:
# First we transpile the GHZ-state for the actual device
qc_transpiled = transpile(simple_ghz, backend, initial_layout=initial_layout)

これで127量子ビットデバイスでジョブを実行する準備が整いました。将来、簡単に見つけられるように、ここにタグを追加しておきましょう。

ジョブの実行には、他の人がどれだけジョブを実行しようとしているかにもよりますが、ある程度の時間がかかるでしょう。今日はもうコーヒーを飲まれましたか？それとも紅茶派ですか？Netflixで素晴らしい新番組があるそうですね。

In [ ]:
# Now we can run the job
# We add memory=true to be easier able to analyse  how good the result were and the tags to make it easier to find it later.
job = backend.run(
    qc_transpiled,
    shots=number_of_shots,
    memory=True,
    job_tags=["ghz_state", "spring_challenge"],
)

job_id = job.job_id()
print(job_id)

後で戻ってきて、ジョブを検索したい場合、IBM Quantum Computing Homepageで、上で使用したタグで見つけることができます。

あなたのjobの名前（id）をコピーして、下のセルの *job_id* をあなたのものに置き換えてください。job IDは次のようなものです： *ch36cf1pleju56fajrqg*

または、上記のセルのjob_idを直接使用することもできます。

In [ ]:
# Change job id to the id of your previous submitted job something like "ch36cf1pleju56fajrqg"
# You only need to run this if you come back at a later time
job = provider.backend.retrieve_job(job_id)
job.status()

In [ ]:
# Getting the data of the job for testing
data = job.result().get_memory()

## Step 4: GHZ状態のテスト

あなたはやり遂げました！127量子ビットマシンでGHZ状態の回路を走らせました。すごいですね!

さて、GHZ状態の結果が出たので、次はそれを検証してみましょう。エラー緩和やエラー訂正は一切していませんので、まだまだ結果に改善が期待できます。

結果がどの程度「良い」かを調べるには、さまざまな方法があります。生の結果を見ることもできますが、GHZ状態で使用される量子ビットは隣り合っていないので、少し煩わしいかもしれません。品質や、GHZ状態のエラーについて教えてくれる機能があれば便利です。

### Exercise 3

GHZ状態をテストするための関数を作成してください。

Challengeの作成者として、私たちは、あなたが結果をどのようにテストしたいのか、ここでいくつかの自由を与えることにしました。 **あなたにとって** 理にかなった方法を見つけてください。関数の _出力が低いほど、_　GHZ状態が _良好である_ ことを示すことが重要です。

*ヒント：この関数は作成したデータをテストするために作成する必要があるため、GHZ状態の一部である量子ビットのみをテストする必要があります。*


In [ ]:
# A function to test the quality of a GHZ-state. The lower the better
def test_ghz(data):
    ghz_qubits = [
        0,
        2,
        4,
        6,
        8,
        10,
        12,
        18,
        20,
        22,
        24,
        26,
        28,
        30,
        32,
        37,
        39,
        41,
        43,
        45,
        47,
        49,
        51,
        56,
        58,
        60,
        62,
        64,
        66,
        68,
        70,
        75,
        77,
        79,
        81,
        83,
        85,
        87,
        89,
        94,
        96,
        98,
        100,
        102,
        104,
        106,
        108,
        114,
        116,
        118,
        120,
        122,
        124,
        126,
    ]

    ####### your code goes here #######


test_ghz(data)

これで、あなたが設計した関数に基づき、結果を評価する数字ができました。

先に進む前に、自分のやり方について考えてみましょう。
- 本当に良い状態の時の数値はどうでしょうか？
- 本当に悪い状態の場合はどうでしょうか？
- 「悪い状態」とはどのような状態なのでしょうか？
- どうすれば、何を言っているのか理解しやすい関数を設計できるのか？

これらの疑問は、IBMの研究者が日々考えていることであり、私たちのデバイスやサービスを可能な限り強く、わかりやすくするために、非常に高いモチベーションを保ち続けています。もし、あなたがこのような仕事に興味をお持ちなら、ぜひ教えてください！

それでは、テスト機能をテストして、それがすべきことをするかどうか見てみましょう。エラーの少ない良い状態には低い結果を、エラーの多い状態には高い結果を与えるべきであることを忘れないでください。

In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5c

# Since we test here a function, we do not need brackets after test_ghz, since the input is the function
grade_ex5c(test_ghz)


さて、問題は、どうすれば改善できるかということです。

まず、回路のDepthをもっと小さくしてみることです。トランスパイルされた回路のDepthを見てみましょう：


In [ ]:
qc_transpiled.depth()

トランスパイルプロセスで、Depthが2倍以上大きくなっていることがわかります。

このDepthがどこから来たのか、その様子をみてみましょう。


In [ ]:
qc_transpiled.draw()

アダマールゲートとCXゲートが、他のゲートにトランスパイルされていることがわかります。IBMの量子デバイスはどれもアダマールゲートを直接サポートしていないので、アダマールゲートについては、驚くべきことではありません。アダマールゲートは常に3つの回転ゲートで構成されます。一方、ご覧のECRゲートは新しいもので、一部のデバイスでしか使われていません。これもCXゲートに似たエンタングルゲートですが、仕組みが少し違います。CXゲートは、ECRゲート１つとローカル回転ゲートだけで作ることができます。

ECRゲートの詳細については、こちらをご覧ください.
: https://qiskit.org/documentation/stable/0.39/stubs/qiskit.circuit.library.ECRGate.html 


Depthを減らすことは、エラー率を最小化するための最良の方法であることが多いです。ECRゲートが代わりに使われることがわかったところで、もっと良いDepthの回路を作ることはできないでしょうか？

その疑問や、GHZ状態をより良くするための他の方法を探ることに興味がある方は、どうぞ先をお読みください。このLabの次の部分は、ボーナス・パートです。この後にExerciseはありません。これは、エラー訂正をどのように使うことができるかを示すための短い展望であると同時に、エラー訂正がなぜ難しいかを示すためのものです。


続けるにしても、続けないにしても、自分に拍手を送ってあげてください： あなたは今、IBM Quantum Challenge: Spring 2023を完了したところです！Discordで他のみんなに自慢してください。Discordでお会いしましょう！

# ボーナス: エラー訂正への道

エラー訂正は、今でも活発で重要な研究テーマです。そのため、実機でのエラー訂正は、今回のGHZ状態の例のような単純なケースであっても、それほど簡単ではありません。このページでは、どのような方法が考えられるかについて、いくつかのガイダンスやヒントを提供しながら、独自のアイデアを生み出す機会を提供したいと思います。



## Step 1: 実際の状態はどの程度よかったのか？

エラーを修正する前に、まず状態がどの程度良かったかを知る必要がありますが、これさえも思ったほど簡単ではありません。

あなたのGHZ状態がどれだけ優れているかをテストする最も簡単な方法は、偶数量子ビットのうち、どれだけが多数派と異なる結果を得たかを調べることです。この方法は、上記で使用した方が多いのではないでしょうか？

しかし、これは必ずしもエラーの発生回数と相関があることを意味するものではありません。そして、GHZの良し悪しを知るには、エラーの発生数の方が重要であるとも言えます。


**例 A** : この考えを説明するために、例えば、10個の一列に並んだ量子ビットで、GHZ状態を形成するとします：

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9

ここで、読み出しではなく、量子ビット4と量子ビット5の間でエンタングルゲートを適用したときにエラーが発生したとすると、出力は次のようになります：

0000011111 

これは、「何個の量子ビットが同じ結果を出すか」という測定において、最悪の結果となります。

このため、より多くのエラーが発生するようにすれば、「何個の量子ビットが同じ結果を持つか」をより正確に測定できる可能性がありますが、ほとんどの場合、その状態はあまり役に立たなくなります。

もし上記のような状態が作られ、量子ビットの間で(機能する)スタビライザー測定が行われていたとしたら、量子ビット4と5の間の測定値が1になるだけです。そして、このエラーを修正するために、量子ビット5-9を反転させることが可能です。

あなたのケースでエラーがどのように見えるかを知るには、実際のデバイスで実行した結果である生データを見て、GHZ状態をどのように構築したかを考慮するのが良いアイデアかもしれません。

**例A** の場合、次のようなエラー訂正が考えられます：

In [ ]:
# Simple idea for Example A find where the error happened and flip accordingly
def correct_ghz():
    qc = QuantumCircuit(quantum_register, classical_register)

    with qc.if_test((classical_register[9], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[8], 1)):
        qc.x(quantum_register[8])

    with qc.if_test((classical_register[8], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[8])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[7])

    # ...

    # qc.barrier()
    return qc


correcting_circuit = correct_ghz()

そして、もし今、私たちが上記のコードを私たちの54GHZのケースに適応させるなら、すべてを一緒に構築することができます：

In [ ]:
# Everything together corrected

error_corrected_ghz = (
    ghz_circuit.compose(unentangle_circuit)
    .compose(stabilizer_circuit)
    .compose(correcting_circuit)
    .compose(measure_circuit)
)

## Step 2: なぜ単一のスタビライザーだけでは不十分なのか

エラー訂正のExerciseでは、スタビライザー量子ビットの測定から、1ビット、またはビット列全体を使用して、その演算を条件付けることができることを確認しました。

スタビライザー測定は54ビットなので、ビット列全体を条件付けに使うのは現実的ではありません。なぜなら、2^54個の異なるビット列を考慮する必要があるからです。

一方、1ビットだけを考慮する場合は、不可能であるか、または過度に複雑になる可能性があります。


**例**: 再び10個の量子ビットを一列に接続し、それらでGHZ状態を形成します：

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9

ここで、測定結果として0000010000が得られたと仮定します。

この場合、スタビライザー測定では、1の右側と1の左側がともに値1になります。

つまり、この場合、1つのスタビライザーを見ただけでは、5番の量子ビットを反転させればよい（他の量子ビットは反転させない）ことを知るには不十分であることがわかります。ただし、いくつかの量子ビットを何度も反転させてもよいという場合はこの限りではありませんが、それは理想的ではありません。

そこで、この場合、スタビライザーの測定結果に関数を適用し、その結果を動的回路の条件として使用できれば理想的です。

これは可能ですが、QiskitではなくOpen QASM3のコードで行う必要があり、このExerciseでは取り上げません。
Open QASM3によるDynamic Circuitについてもっと知りたい場合は、ここで情報を得ることができます： 

https://quantum-computing.ibm.com/services/resources/docs/resources/manage/systems/dynamic-circuits/Dynamic-Circuits-basics-with-OpenQASM3

以下は、Open QASM3のコードの簡単な例です：

In [ ]:
from qiskit import qasm3, QuantumCircuit, transpile

# Creating a bell circuit
qc_bell = QuantumCircuit(2, 2)
qc_bell.h(0)
qc_bell.cx(0, 1)
qc_bell.measure(0, 0)
qc_bell.measure(0, 1)

# Transpiling it for our device (as above it does not have the H- and CX- Gates)
qc_bell = transpile(qc_bell, backend)

# Generate qasm3 code before we can print it
exporter = qasm3.Exporter(
    includes=[], disable_constants=True, basis_gates=backend.configuration().basis_gates
)
print(qasm3_bell := exporter.dumps(qc_bell))

# Draw a circuit as comparison
qc_bell.draw(output="mpl", idle_wires=False)

## Step 3: エラーを増やさないためにはどうするか？


エラーが発生する原因はさまざまです。時間経過によるデコヒーレンスや、CX-Gateのようなエンタングルゲートで発生するエラーは、そのうちの2つです。この2つはスタビライザーにも当てはまるので、本末転倒にならないように、エラーを修正しようとすることでノイズを増やさないようにする必要があります。将来的には、一般的なエラーレートがさらに下がるので、それほど問題にはならないかもしれませんが、それでも考慮しなければならないことです。

このことは、私たちにとってどのような意味を持つのでしょうか？

スタビライザーを作成し、測定したい時について考えてみましょう。次の量子ビットのエンタングルに使われなくなった後、直接エンタングルを解くのでしょうか？ 回路全体がエンタングルされるまで待つのでしょうか？(これはもちろん、どのような種類のエラーを修正したいかによります。）また、スタビライザーをリセットして、後で新しいものを作る（同じ量子ビットを使う）、エラーを修正するために2つのフェーズを使う、ということも考えられます。

そして、（スタビライザーを通して）量子ビットをエンタングルすることでエラーが発生する確率はどれくらいなのかを考え、スタビライザーをアンエンタングルすることでエラーが発生する確率と比較することができます。つまり、スタビライザーがあることで、かえって悪化しているのかも？

その価値はいつあるのでしょうか？以下のコードで遊んでみて、印象やアイデアを得ることができます。ただし、これは将来、エラーレートがさらに低下したときに変更される可能性があります。

In [ ]:
# All the probabilities here only consider errors introduced by the CX gate and assumes they are bit flip errors.

# Probability for a single CX gate
p1 = 0.01
# Probability that there is an error after 2 CX gates (going through stabilizer)
p2 = p1 * (1 - p1) + (1 - p1) * p1
# Probability that the stabilizer shows something wrong even though it is correct
p3 = p1 * p1 + (1 - p1) * (1 - p1) * p1

print("Probability of a single cx having an error: {}".format(p1))
print("Probability of having an error after 2 cx: {:.4f}".format(p2))
print("Probability of the stabilizer showing a non existent error: {:.4f}".format(p3))

## Step 4: 何ができるでしょうか？

あなたが試せる可能性はたくさんあります：



- 複数のスタビライザーを使って何か良い論理的な機能を見つけ、QASM3を使ってみるのも良いでしょう。

- スタビライザーをリセットして再利用することを考えることができます。

- 例Aのようなシンプルな方法をうまく使えないか試すことができます（測定するタイミングを変えるなど）。

- 自分自身のアイデアを見つけることができます！クリエイティブになりましょう！


いずれも重要なのは、やみくもに量子コンピュータで試すのではなく、最初のGHZ状態を走らせたデータを使って、まず自分の考えを検証してみることです。

上記でテストデータを作成しました： GHZ状態の出力と、スタビライザーの出力があります。

あなたのアプローチが後処理としてうまくいかないのであれば、実機で動作させれば、すべての演算にさらなるエラーレートが発生するため、うまくいかないでしょう。

手持ちのデータで理論的に動作するアルゴリズムができたら（テストによりGHZ状態が良くなったら）、以下のコードを使って、エラー訂正を含む回路全体を実機で動かしてみることができるようになります。

*ヒント：上記のerror_correction関数を使用し、上記のerror_corrected_ghzも生成することを確認してください。

In [ ]:
# First we transpile the GHZ-state for the actual device
qc_corrected_transpiled = transpile(
    error_corrected_ghz, backend, initial_layout=initial_layout
)

# Now we can run the job
job_corrected = backend.run(
    qc_corrected_transpiled,
    dynamic=True,
    shots=number_of_shots,
    memory=True,
    job_tags=["dynamic", "spring_challenge"],
)

job_id = job_corrected.job_id()
print(job_id)

In [ ]:
job_corrected = provider.retrieve_job(job_id)
job_corrected.status()

In [ ]:
# And get the results back
counts_corrected = job_corrected.result().get_counts()